In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazon-fine-food-reviews/hashes.txt
/kaggle/input/amazon-fine-food-reviews/Reviews.csv
/kaggle/input/amazon-fine-food-reviews/database.sqlite


In [2]:

import torch
from torch import nn
from torch.nn import functional as F
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.nn.functional import cross_entropy  # for classification loss


2025-06-28 14:02:35.019262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751119355.242260      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751119355.305062      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
data = pd.read_csv('/kaggle/input/amazon-fine-food-reviews/Reviews.csv');

In [4]:
data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [5]:
new_data = data.sort_values(by = 'Score')
        

In [6]:
grouped_data = data.groupby('Score');

In [7]:
data  = grouped_data.apply(lambda x: x.sample(min(10000, len(x))).reset_index(drop=True))

/tmp/ipykernel_19/251621599.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data  = grouped_data.apply(lambda x: x.sample(min(10000, len(x))).reset_index(drop=True))


In [8]:
print(data['Score'].value_counts())

Score
1    10000
2    10000
3    10000
4    10000
5    10000
Name: count, dtype: int64


In [9]:
data.iloc[1, -1]

'I would never buy this cocoa ever again. this was by far the foulest substance to ever cross my lips. DONT BUY THIS<br />no stars'

In [10]:
dataset = data.sample(len(data)).reset_index(drop=True)

In [11]:
dataset = dataset.iloc[:, [-1,-4]]

In [12]:
dataset

,Text,Score
0,"I have 2 medium sized dogs, a siberian husky w...",3
1,This stuff tastes great to me but is a bit too...,4
2,"Tried this product, the coffee was good, the f...",3
3,These lollipops are not as pictured. They came...,1
4,I am of the opinion that VIA Columbian is pret...,2
...,...,...
49995,"This flavor has an overwhelming peanut, i.e. p...",2
49996,Take captain crush and jam a fake 'fudge' cent...,2
49997,"Should not be labeled ""Key Lime Juice"" because...",1
49998,Good size pieces of freeze dried liver. Few s...,5


In [13]:
inputs = [];

for row in np.array(dataset):
    if len(row)>300:
        continue
    inputs.append(row[0]);
    
    

In [14]:
tokenizer = Tokenizer();
tokenizer.fit_on_texts(inputs);

In [15]:
maxx = len(tokenizer.word_index)

In [16]:
maxx

45466

In [17]:
tokenized_inputs  = tokenizer.texts_to_sequences(inputs)

In [18]:
max_len  = max([len(i) for  i in inputs])

In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_padded = pad_sequences(tokenized_inputs ,maxlen=300, padding='post')         

In [20]:
final_input = torch.tensor(X_padded, dtype=torch.float32 )

In [21]:
dataset.iloc[:,-1]

0        3
1        4
2        3
3        1
4        2
        ..
49995    2
49996    2
49997    1
49998    5
49999    5
Name: Score, Length: 50000, dtype: int64

In [22]:
final_ouput = torch.tensor(dataset.iloc[:,-1], dtype=torch.float32)

In [23]:
class LSTMclassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMclassifier, self).__init__()
        self.embedding = nn.Embedding(maxx+1, 300)
        self.lstm  =  nn.LSTM(300,hidden_size, batch_first = True);
        self.fc = nn.Linear(hidden_size, num_classes);

    def forward(self, X):
       X = self.embedding(X)
       out, (hn,cn)= self.lstm(X);
       output = hn[-1];
       return self.fc(output)
       
       

In [24]:
final_input = final_input.long()
final_ouput = final_ouput.long()
from sklearn.model_selection import train_test_split


# Example: X_tensor (N, D), y_tensor (N,)
X_train, X_test, y_train, y_test = train_test_split(
   final_input, final_ouput, test_size=0.2, random_state=42
)

# Wrap them in TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Create loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [25]:


import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMclassifier(input_size=300, hidden_size=15, num_classes=6).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(5):
    model.train()
    for batch_x, batch_y in train_loader:
        # Move data to GPU
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        logits = model(batch_x)
        loss = criterion(logits, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Save model
    torch.save(model.state_dict(), f"model_epoch_{epoch+1}.pt")
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

    model.eval()
    test_loss = 0
    corr = 0

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            # Move test data to GPU
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            outputs = model(batch_x)
            loss = criterion(outputs, batch_y)
            test_loss += loss.item()

            # Sample predictions
            pred = torch.argmax(F.softmax(outputs, dim=-1), dim=-1).squeeze()

            corr += (pred == batch_y).sum().item()

    accuracy = corr / len(test_loader.dataset)  # fixed this line
    avg_test_loss = test_loss / len(test_loader)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Average Test Loss: {avg_test_loss:.4f}")


Epoch 1, Loss: 1.6109
Accuracy: 0.2039
Average Test Loss: 1.6143
Epoch 2, Loss: 1.6505
Accuracy: 0.2050
Average Test Loss: 1.6149
Epoch 3, Loss: 1.6114
Accuracy: 0.2292
Average Test Loss: 1.6152
Epoch 4, Loss: 1.6093
Accuracy: 0.2296
Average Test Loss: 1.6134
Epoch 5, Loss: 1.6467
Accuracy: 0.2279
Average Test Loss: 1.6179


In [26]:

import torch.nn.functional as F


# # Load model
# model.load_state_dict(torch.load("model_epoch_3.pt"))
# model.eval()

# Input string
input = X_test.to(device);

# Inference
with torch.no_grad():
    # tokenized_in =tokenizer.texts_to_sequences([input]) # Make sure this returns a torch.Tensor
    output = model(torch.tensor(input, dtype=torch.long).to(device))      # forward pass
    pred = torch.argmax(F.softmax(output, dim=-1), dim=-1)  # torch.argmax, not np.argmax
    print(pred)
    Accu_count = (pred.to(device) == y_test.to(device)).sum().item() / len(y_test)
    # _,counts = torch.unique(torch.tensor(Accu_count).long(), return_counts = True)
    print(Accu_count)
   
    


    


tensor([5, 5, 5,  ..., 1, 5, 5], device='cuda:0')
0.2279


/tmp/ipykernel_19/4018363629.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output = model(torch.tensor(input, dtype=torch.long).to(device))      # forward pass


In [27]:
#Reached just 24% accuracy,so lets try pretrained models

In [28]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased",  
    num_labels=6
)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")



config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [29]:
import pandas as pd

data = pd.read_csv('/kaggle/input/amazon-fine-food-reviews/Reviews.csv');

In [30]:
import numpy as np

arr = np.zeros(6);
for i in data.iloc[:1000, -4]:
    arr[i]= arr[i]+1;

print(arr)
    

[  0.  98.  47.  75. 138. 642.]


In [31]:
new_data = data.sort_values(by = 'Score')
grouped_data = data.groupby('Score');
data  = grouped_data.apply(lambda x: x.sample(min(10000, len(x))).reset_index(drop=True))
        

/tmp/ipykernel_19/988022090.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data  = grouped_data.apply(lambda x: x.sample(min(10000, len(x))).reset_index(drop=True))


In [32]:
data

Id   ProductId              UserId  \
Score                                                
1     0     331114  B001UJCKCG      A1OEXW2VO7E8SC   
      1     465854  B001YJ9LME      A1PJK9DTHQI2W7   
      2     366940  B001AIQP8M      A2VLQO1ZY057O2   
      3     376388  B000EM2PHK      A124VGEBZLR6E7   
      4     516182  B008I1XPKA  #oc-R1SA5IJKJKB5NX   
...            ...         ...                 ...   
5     9995  404130  B000LKWUXE      A1FDZLHWVR70VK   
      9996  326893  B0009F3SEQ       AWPN47SSWK1JV   
      9997  236040  B008LU5R3Q      A12HU8KT4O3UZV   
      9998  189566  B001EO5Q64      A1IX7KII2PYKTC   
      9999  436501  B000FFLTD2      A2J6CJSTGJYXEV   

                                      ProfileName  HelpfulnessNumerator  \
Score                                                                     
1     0                                      Gina                     0   
      1                   TheContinuingSaga "TCS"                     1   
      2                               kimmyp "kp"                     0   
      3                            William Lefler                     3   
      4                 PALA "Reluctant Reviewer"                     3   
...                                           ...                   ...   
5     9995  LMK-A "bargain sleuth, freebie freak"                     2   
      9996                             K. Nordhus                     1   
      9997                                    LDA                     0   
      9998                                    KSL                     0   
      9999                                   Vrod                     0   

            HelpfulnessDenominator  Score        Time  \
Score                                                   
1     0                          0      1  1334016000   
      1                          1      1  1263081600   
      2                          1      1  1298073600   
      3                         10      1  1200614400   
      4                          5      1  1346284800   
...                            ...    ...         ...   
5     9995                       2      5  1238716800   
      9996                       1      5  1258156800   
      9997                       1      5  1318550400   
      9998                       0      5  1350691200   
      9999                       0      5  1296604800   

                                               Summary  \
Score                                                    
1     0                                DO NOT PURCHASE   
      1                    STALE AND DISAPPOINTED.....   
      2             would give it no stars if possible   
      3                  Where did the sugar come from   
      4                     Don't get the French Roast   
...                                                ...   
5     9995  Best Salt & Vinegar that I've ever tasted!   
      9996                                 Works Great   
      9997                     Great product & Company   
      9998                          Kerry's Comments -   
      9999                                    Fabulous   

                                                         Text  
Score                                                          
1     0     This was gross. No other way to describe it. T...  
      1     This item arrived to me STALE! I was very, ver...  
      2     I took one bite and gagged.  Tasted like chemi...  
      3     This is the most awful french onion soup I hav...  
      4     It's weak and has very little flavor.  It's a ...  
...                                                       ...  
5     9995  These are amazing! They're intensely craveable...  
      9996  I have the worst cramps every month. By this I...  
      9997  <a href="http://www.amazon.com/gp/product/B004...  
      9998  This stuff "Nutiva Organic Extra Virgin Coconu...  
      9999  I had this soup and immediately logged on to A...  

[50000 

In [33]:
dataset = data.sample(len(data)).reset_index(drop=True)

In [34]:
dataset

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,388060,B007RTR8UM,A2UM324B00HBC8,"CLR ""Celestial""",0,0,4,1340150400,Very good conditioner -- just short of great,"UPDATE: The longer I use this conditioner, the..."
1,513769,B001EO5YM0,A2IYTXOM5CVQWF,"Blade ""Blade""",0,0,4,1326412800,"Great soup, its like the adult version of cup-...","Great soup, its like the adult version of cup-..."
2,408821,B000KGW2BY,A21LAL2YC204AW,Mick the Ginge,0,1,1,1313020800,Caution - Item ordered in June delivered with ...,Double check the best before date when your ch...
3,113618,B000E671Z2,AXNZNHZJB38LN,Dummy,0,0,1,1346889600,celestial tea (peach) cold brew,"No real flavor,used up two 4 bags and still no..."
4,406132,B00122BKI6,A2FSN7CSC3TWN,Molly,0,0,4,1342742400,Pods for Senseo Coffee Maker,Pods are individually wrapped and dispensed ve...
...,...,...,...,...,...,...,...,...,...,...
49995,545207,B000EHOZRI,A11GUG7T85VS95,BobbyB,0,0,2,1348012800,"Very, very dry turkey jerky, if you like Trade...",I ordered the 6 pack of Shelton's Turkey Jerky...
49996,9902,B00448RTKS,A1H3FJR4GY5B46,Cleo,0,0,1,1296086400,Poor Kitty,"Bottom line, this made my cat projectile vomit..."
49997,437828,B005A1LGIY,A2UQGGK8VJPUH5,I. Zawilski,0,0,3,1322870400,It's Okay,I tried both the Margarita and Appletini flavo...
49998,108256,B007TJGY46,A31ZA2TIIRGF6Y,"R. Grabowski ""BeeGee""",0,0,4,1268352000,love the reduced packaging,Excellent bold coffee and I love that it is al...


In [35]:
dataset = dataset.iloc[:, [-1,-4]]

In [36]:
dataset

,Text,Score
0,"UPDATE: The longer I use this conditioner, the...",4
1,"Great soup, its like the adult version of cup-...",4
2,Double check the best before date when your ch...,1
3,"No real flavor,used up two 4 bags and still no...",1
4,Pods are individually wrapped and dispensed ve...,4
...,...,...
49995,I ordered the 6 pack of Shelton's Turkey Jerky...,2
49996,"Bottom line, this made my cat projectile vomit...",1
49997,I tried both the Margarita and Appletini flavo...,3
49998,Excellent bold coffee and I love that it is al...,4


In [37]:
def process(dataset):
    # inputt  = list(dataset['Text']);
    inputs = tokenizer(
        dataset['Text'],
        max_length=300,
        truncation=True,
        padding="max_length"
    )
    inputs['labels'] = dataset['Score']
    return inputs

In [38]:
from datasets import Dataset
tokenized_data = Dataset.from_pandas(dataset).map(process)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [39]:
# Save tokenized dataset
tokenized_data.save_to_disk("/kaggle/working/tokenized_dataset")


Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [40]:



split = tokenized_data.train_test_split(test_size=0.2)
train_data = split["train"]
val_data = split["test"]


# Keep format
columns = ["input_ids", "attention_mask", "labels"]
train_data.set_format(type="torch", columns=columns)
val_data.set_format(type="torch", columns=columns)


In [41]:
dataset.columns

Index(['Text', 'Score'], dtype='object')

In [42]:
train_data = train_data.remove_columns(dataset.columns)

In [43]:
val_data  = val_data.remove_columns(dataset.columns)

In [44]:
train_data

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 40000
})

In [45]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=f"rating",
    # evaluation_strategy="epoch",
    logging_strategy="steps",
    learning_rate=5.6e-5,
    run_name=None,                   # Don't need a run name
    report_to="none",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    # predict_with_generate=True,
    logging_steps=16,
    push_to_hub=False,
     disable_tqdm=False,
    # evaluation_strategy= "epoch"
)


In [46]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn

In [47]:
from transformers import Trainer
import evaluate
accuracy = evaluate.load('accuracy');

def compute(eval_pred):
    pred, labels = eval_pred
    predss = pred.argmax(axis = -1)
    return accuracy.compute(predictions = predss, references = labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute,
    tokenizer=tokenizer

)

/tmp/ipykernel_19/2072615767.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [48]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
16,1.728000
32,1.652000
48,1.522600
64,1.557400
80,1.356000
96,1.173400
112,1.396900
128,1.200100
144,1.181900
160,1.123400


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=7500, training_loss=0.735403136253357, metrics={'train_runtime': 4282.7943, 'train_samples_per_second': 28.019, 'train_steps_per_second': 1.751, 'total_flos': 1.8500660496e+16, 'train_loss': 0.735403136253357, 'epoch': 3.0})

In [49]:
trainer.evaluate()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 1.0391818284988403,
 'eval_accuracy': 0.6628,
 'eval_runtime': 121.9094,
 'eval_samples_per_second': 82.028,
 'eval_steps_per_second': 5.127,
 'epoch': 3.0}

In [50]:
import torch

# Ensure model is on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainer.model.to(device)

# Tokenize input and move tensors to GPU
inputs = tokenizer(
    "this is very good",
    max_length=300,
    truncation=True,
    padding="max_length",
    return_tensors="pt"
).to(device)

# Run model
with torch.no_grad():
    output = trainer.model(**inputs)
    logits = output.logits

# Print logits and predicted class
print("Logits:", logits)
print("Predicted class:", torch.argmax(logits, dim=-1))


Logits: tensor([[-3.5048, -2.2341, -2.4127, -0.2368,  3.3266,  3.5758]],
       device='cuda:0')
Predicted class: tensor([5], device='cuda:0')
